**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/analytic-functions).**

---


# Introduction

Here, you'll use window functions to answer questions about the [Chicago Taxi Trips](https://www.kaggle.com/chicago/chicago-taxi-trips-bq) dataset.

Before you get started, run the code cell below to set everything up.

In [1]:
# Get most recent checking code
!pip install -U -t /kaggle/working/ git+https://github.com/Kaggle/learntools.git
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex2 import *
print("Setup Complete")

  Cloning https://github.com/Kaggle/learntools.git to /tmp/pip-req-build-8czfzjm8
  Running command git clone --filter=blob:none --quiet https://github.com/Kaggle/learntools.git /tmp/pip-req-build-8czfzjm8
  Resolved https://github.com/Kaggle/learntools.git to commit 0b915896f0a90bedb3b94d36398a53cd31c402d7
  Preparing metadata (setup.py) ... done
  Created wheel for learntools: filename=learntools-0.3.4-py3-none-any.whl size=268968 sha256=ac9f579d36d170db1106e7f36d74b1a63b9214e9cf07cd0ea778bebf8b00bbf8
  Stored in directory: /tmp/pip-ephem-wheel-cache-yoeel4ui/wheels/2f/6c/3c/aa9f50cfb5a862157cb4c7a5b34881828cf45404698255dced
Successfully built learntools
Using Kaggle's public dataset BigQuery integration.
Setup Complete


The following code cell fetches the `taxi_trips` table from the `chicago_taxi_trips` dataset. We also preview the first five rows of the table.  You'll use the table to answer the questions below.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,6d0b0189616a2e04d709cf26b71ed322ab1e486d,22b403ba1c4de25cb19049e9c08b6d1339a9eea3ec06f9...,2018-12-31 14:15:00+00:00,2018-12-31 14:45:00+00:00,1636,12.0,NaN,NaN,42,27,...,0.0,32.00,Cash,Flash Cab,41.778877,-87.594925,POINT (-87.5949254391 41.7788768603),41.878914,-87.705897,POINT (-87.7058971305 41.8789144956)
1,796024ad1ee09418fb6e5ed72c89b6c872a3fd75,9e0a7baa1c29a7b1e0b90910c5a718651267de8b208044...,2018-12-03 07:30:00+00:00,2018-12-03 07:45:00+00:00,743,1.2,NaN,NaN,15,15,...,0.0,8.50,Cash,Flash Cab,41.954028,-87.763399,POINT (-87.7633990316 41.9540276487),41.954028,-87.763399,POINT (-87.7633990316 41.9540276487)
2,49f3cefdb156465dc7e103c6213242f35bd87508,7985168ea616aa1c4437d4ebe434ea81f011949cd039dd...,2018-10-19 08:00:00+00:00,2018-10-19 08:15:00+00:00,218,0.4,NaN,NaN,11,15,...,0.0,4.75,Cash,Flash Cab,41.978830,-87.771167,POINT (-87.771166703 41.9788295262),41.954028,-87.763399,POINT (-87.7633990316 41.9540276487)
3,7bf814d54874dc52487611a84ecf06c687f83ffb,54270f3cd6c3f5c7377ece5ea2403dcbf079610ddf4716...,2018-10-02 16:15:00+00:00,2018-10-02 16:30:00+00:00,619,1.8,1.703184e+10,1.703107e+10,8,7,...,0.0,11.25,Credit Card,Flash Cab,41.904935,-87.649907,POINT (-87.6499072264 41.9049353016),41.921778,-87.651062,POINT (-87.6510618838 41.9217781876)
4,3df3fdbcec4c7933c0b6ea1b522bc5776c5cc42d,71898dd7fdff4620c76bd9ca9b02f388830ce768f929fe...,2018-12-31 17:15:00+00:00,2018-12-31 17:15:00+00:00,42,0.1,NaN,NaN,19,19,...,1.0,4.50,Cash,Flash Cab,41.927261,-87.765502,POINT (-87.7655016086 41.9272609555),41.927261,-87.765502,POINT (-87.7655016086 41.9272609555)


# Exercises

### 1) How can you predict the demand for taxis?

Say you work for a taxi company, and you're interested in predicting the demand for taxis.  Towards this goal, you'd like to create a plot that shows a rolling average of the daily number of taxi trips.  Amend the (partial) query below to return a DataFrame with two columns:
- `trip_date` - contains one entry for each date from January 1, 2016, to March 31, 2016.
- `avg_num_trips` - shows the average number of daily trips, calculated over a window including the value for the current date, along with the values for the preceding 3 days and the following 3 days, as long as the days fit within the three-month time frame.  For instance, when calculating the value in this column for January 3, 2016, the window will include the number of trips for the preceding 2 days, the current date, and the following 3 days.

This query is partially completed for you, and you need only write the part that calculates the `avg_num_trips` column.  Note that this query uses a common table expression (CTE); if you need to review how to use CTEs, you're encouraged to check out [this tutorial](https://www.kaggle.com/dansbecker/as-with) in the [Intro to SQL](https://www.kaggle.com/learn/intro-to-sql) course.

In [5]:
# Fill in the blank below
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp >= '2016-01-01' AND trip_start_timestamp < '2016-04-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
                               ) AS avg_num_trips
                      FROM trips_by_day
                      """
# Check your answer
q_1.check()

,trip_date,avg_num_trips
0,2016-02-04,80368.285714
1,2016-01-20,84509.142857
2,2016-01-26,82071.285714
3,2016-03-17,96865.428571
4,2016-03-11,99220.000000


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Can you separate and order trips by community area?

The query below returns a DataFrame with three columns from the table: `pickup_community_area`, `trip_start_timestamp`, and `trip_end_timestamp`.  

Amend the query to return an additional column called `trip_number` which shows the order in which the trips were taken from their respective community areas.  So, the first trip of the day originating from community area 1 should receive a value of 1; the second trip of the day from the same area should receive a value of 2.  Likewise, the first trip of the day from community area 2 should receive a value of 1, and so on.

Note that there are many numbering functions that can be used to solve this problem (depending on how you want to deal with trips that started at the same time from the same community area); to answer this question, please use the **RANK()** function.

In [6]:
# Amend the query below
trip_number_query = """
                    WITH trips_with_rank AS
                    (
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK() OVER (PARTITION BY pickup_community_area, DATE(trip_start_timestamp) ORDER BY trip_start_timestamp) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2013-10-03'
                    )
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        trip_number
                    FROM trips_with_rank
                    """

# Check your answer
q_2.check()

,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,65.0,2013-10-03 12:30:00+00:00,2013-10-03 12:30:00+00:00,1
1,65.0,2013-10-03 12:45:00+00:00,2013-10-03 12:45:00+00:00,2
2,58.0,2013-10-03 01:15:00+00:00,2013-10-03 01:30:00+00:00,1
3,58.0,2013-10-03 05:30:00+00:00,2013-10-03 05:45:00+00:00,2
4,58.0,2013-10-03 06:00:00+00:00,2013-10-03 06:00:00+00:00,3


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_2.hint()
#q_2.solution()

### 3) How much time elapses between trips?

The (partial) query in the code cell below shows, for each trip in the selected time frame, the corresponding `taxi_id`, `trip_start_timestamp`, and `trip_end_timestamp`. 

Your task in this exercise is to edit the query to include an additional `prev_break` column that shows the length of the break (in minutes) that the driver had before each trip started (this corresponds to the time between `trip_start_timestamp` of the current trip and `trip_end_timestamp` of the previous trip).  Partition the calculation by `taxi_id`, and order the results within each partition by `trip_start_timestamp`.

Some sample results are shown below, where all rows correspond to the same driver (or `taxi_id`).  Take the time now to make sure that the values in the `prev_break` column make sense to you!

![first_commands](https://storage.googleapis.com/kaggle-media/learn/images/qjvQzg8.png)

Note that the first trip of the day for each driver should have a value of **NaN** (not a number) in the `prev_break` column.

In [7]:
# Fill in the blanks below
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp) 
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2013-10-03' 
                   """

# Check your answer
q_3.check()

,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,05001a5e70a7a8f38a2e4d2ee8a1e0fb87a20857413b44...,2013-10-03 13:30:00+00:00,2013-10-03 14:00:00+00:00,165.0
1,0520dba051796dd5a5a6bb274310b504d97f3a3b0b7d18...,2013-10-03 14:00:00+00:00,2013-10-03 14:15:00+00:00,255.0
2,125667cf70469e4ada5ced2973ac5fa8d2dc10745c1267...,2013-10-03 21:00:00+00:00,2013-10-03 21:00:00+00:00,210.0
3,1be1407efca4ff6ac787c8794df57e8eeac031eaf08fca...,2013-10-03 21:15:00+00:00,2013-10-03 21:15:00+00:00,165.0
4,1e5b69c83640bd5225fb3ccc33bc989b859f3e4cdb59f6...,2013-10-03 14:15:00+00:00,2013-10-03 14:15:00+00:00,270.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

# Keep going

Move on to learn how to query **[nested and repeated data](https://www.kaggle.com/alexisbcook/nested-and-repeated-data)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/advanced-sql/discussion) to chat with other learners.*